In [1]:
import numpy as np
from numpy import tanh
import matplotlib.pyplot as plt
import random
from math import sqrt
from model import Model
from rbm import RBM
from rbm_operator import Operator, Sx_, Sy_, Sz_, SzSz_, set_h_Hamiltonian, set_J1_Hamiltonian, set_J2_Hamiltonian

# Simple attempt to implement a variational quantum state calculation 

Reminder: the operators are defined as 
$$S\cdot S = S_x \cdot S_x + S_y \cdot S_y + S_z \cdot S_z = \frac{1}{2} (S_+ \cdot S_- + S_- \cdot S_+) + S_z \cdot S_z$$

where $S_z = \frac{\hbar}{2} \begin{pmatrix} 1 & 0 \\ 0  & -1 \end{pmatrix}$, $S_+ = \hbar \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix} = S_x + S_y$, $S_- = \hbar \begin{pmatrix} 0 & 0 \\ 1 & 0 \end{pmatrix} = S_x - S_y$. A singlet state has energy $-3J/4$ and a triplet state has energy $J/4$

As an example, we start with a very simple J1/J2 Heisenberg spin chain

We define a class "model" that contains a Hamiltonian and a rule for generating spins according to the MSMS algorithm.

We have something to obtain the wavefunction amplitude $\psi_\theta(s)$ from the spin configuration $s$ that depends on some amplitude $\theta$

From  $\psi_\theta(s)$ for the given spin configuration, we can produce an estimate of the energy of the variational ground state.

Then we can perform a stochastic gradient descent based on it.

# Implementation of the RBM class

After having a class of spin and Hamiltonian, consider a wavefunction object, take takes a spin and returns a number. The variational quantum state is defined as 

\begin{align*}
\Psi_M(S;W) = \sum_{h_i} e^{\sum_j a_j \sigma^z_j + \sum_i b_i h_i + \sum_{ij} W_{ij} h_i \sigma^{z}_j}
\end{align*}

Here the free parameters of the models are $a_i, b_, W_{ij}$ and $h_1, \dots, h_M$ represents auxilliary spin variables in the network. The internal spins can be explicited traced out to read

\begin{align*}
\Psi(S;W) = e^{\sum_j a_j \sigma^z_j} \times \Pi_{i=1}^M F_i(S)
\end{align*}

where

\begin{align*}
F_i(S) = 2\cosh\left[ b_i + \sum_J W_{ij} \sigma^z_j\right] =  2\cosh\theta_i
\end{align*}

The object would be an NN (most basic example includes the Carleo RBM, which actually has an analytical form). $a_j$ is the visible layer with the physical spins, $b_i$ is the hidden layer with an arbitrary layer of hidden spins, while $W_{ij}$ is a weight connecting the physical and hidden spins.

I need a function to compute the variational energy. A function to give gradient. And a function to do the gradient descent.


# Operators and derivatives: 

Noting that an operator dcdan be approximated by its local value

\begin{align*}
O_{loc}(s) = \sum_{s, s'}  \left< s\middle| O \middle| s' \right>\frac{\left<s' \middle| \Phi_\theta\right>}{\left<s \middle| \Phi_\theta\right>}
\end{align*}

and an operator's expectation value can be reasonably approximated by 

\begin{align*}
\left<O \right> = \sum_{s} P(s) O_{loc}(s) \approx \frac{1}{M} \sum_{s_i} O_{loc}(s_i)
\end{align*}

Now consider the energy minimization. Let $O_p(s) = \frac{\partial}{\partial \theta_p} \log \left<s\middle|\Psi_\theta\right> = \left<s\middle|O_p \middle| s\right> $,

\begin{align*}
\frac{\partial E(\theta)}{\partial \theta_p} &= 2 \Re \left[ \left\langle E_{\text{loc}}(\mathbf{s}) O^*(\mathbf{s}) \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle O^*(\mathbf{s}) \right\rangle \right] \\
&= 2 \Re \left[ \left\langle (E_{\text{loc}}(\mathbf{s}) - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle) O^*(\mathbf{s}) \right\rangle \right]
\end{align*}

To evaluate the derivatives, note that 

\begin{align*}
O_{a_i} = \frac{\partial}{\partial a_i} \log \left<s\middle|\Psi_\theta\right> &= \sigma_i^z \\
O_{b_j} = \frac{\partial}{\partial b_j} \log \left<s\middle|\Psi_\theta\right> &= \tanh(\theta_j(S)) \\
O_{W_{ij}} = \frac{\partial}{\partial W_{ij}} \log \left<s\middle|\Psi_\theta\right> &= \sigma^z_i\tanh(\theta_j(S))
\end{align*}

It follows that the gradient is given by (up to proportionality factors)

\begin{align*}
\frac{\partial E(\theta)}{\partial a_i}  &= \left[ \left\langle E_{\text{loc}}(\mathbf{s}) \sigma_i^z \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle \sigma_i^z \right\rangle \right] \\
\frac{\partial E(\theta)}{\partial b_j}  &= \left[ \left\langle E_{\text{loc}}(\mathbf{s}) \tanh(\theta_j(s)) \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle \tanh(\theta_j(s)) \right\rangle \right] \\
\frac{\partial E(\theta)}{\partial W_{ij}}  &= \left[ \left\langle E_{\text{loc}}(\mathbf{s}) \sigma^z_i\tanh(\theta_j(s)) \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle \sigma^z_i\tanh(\theta_j(s)) \right\rangle \right]
\end{align*}


Technically, I can use the naive gradient descent. But it tends to have poor convergence because the wavefunction is highly non-linear and correlated. Stchastic reconfiguration is known to have better performance. 

## Sanity check: expectation value of operators 

### Two-site operators (SzSz only, for now)

In [6]:
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(SzSz_(0,0,0,1,model), np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <SzSz|ud__|SzSz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(SzSz_(0,0,0,1,model), np.array([[[0, 1], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|dd__|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(SzSz_(0,0,0,1,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <SzSz|ud__|SzSz> is (-0.25+0j) with standard deviation 0.0
The average expectation  <Sz|dd__|Sz> is (0.25+0j) with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.155+0j))


### One-site operator (Sz makes sense. Unsure if Sx does, but I'll sweep it under the rug for now.)

In [7]:
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sz_(0,0,model), np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <Sz|u___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sz_(0,0,model), np.array([[[0, 1], [1, 0]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|d___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sz_(0,0,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sz|u___|Sz> is (0.5+2.661629416864979e-17j) with standard deviation 3.0814879110195774e-33
The average expectation  <Sz|d___|Sz> is (-0.5+0j) with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.024999999999999984+2.216197069250011e-18j))


In [8]:
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sz_(1,0,model)+Sz_(0,1,model), np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <Sz|u___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sz_(1,0,model)+Sz_(0,1,model), np.array([[[0, 1], [1, 0]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|d___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sz_(1,0,model)+Sz_(0,1,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sz|u___|Sz> is (-1+0j) with standard deviation 0.0
The average expectation  <Sz|d___|Sz> is 0j with standard deviation 0.0
The average expectation for a 2:1 mixed state is 0j)


In [9]:
operator = Operator(model)
for i in range(2):
    for j in range(2):
        operator += Sz_(i, j, model)
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(operator, np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <Sz|u___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(operator, np.array([[[0, 1], [1, 0]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|d___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(operator, batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sz|u___|Sz> is 0j with standard deviation 0.0
The average expectation  <Sz|d___|Sz> is 0j with standard deviation 0.0
The average expectation for a 2:1 mixed state is (1.5150000000000001+0j))


In [10]:
#NB: This assumes that the spin is up for probability 2/3 and down for probabilty 1/3
#If you set them to be the same the spin expectation is 1/2, which checks out for |up>+|dn>
model = Model(2,3)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sx_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation <Sx|u___|Sx> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sx_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation  <Sx|d___|Sx> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(100) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sx_(0,0,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sx|u___|Sx> is (0.27052722383953354-0.3146306741451852j) with standard deviation 0.0
The average expectation  <Sx|d___|Sx> is (0.27052722383953354-0.3146306741451852j) with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.22609919571493195+0.041158037312519546j))


In [11]:
model = Model(2,3)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sy_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation <Sy|u___|Sy> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sy_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation  <Sy|d___|Sy> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sy_(1,0,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sy|u___|Sy> is (0.0055333473245139375-0.6675393938788688j) with standard deviation 1.1103585416259206e-16
The average expectation  <Sy|d___|Sy> is (0.0055333473245139375-0.6675393938788688j) with standard deviation 1.1103585416259206e-16
The average expectation for a 2:1 mixed state is (0.10322278215222992-0.38331483265713673j))


# Implementation of the learning process

Idea:

For each iteration:

Start with a set of weights.

For each weight, initialize the batch of state configurations based on the MCMC. For each state, get the amplitude. The combination gives an estimation of energy. This energy generates an estimation of gradient descent. Change the weight. 



In [61]:
model = Model(2,3)
rbm = RBM(model)
Ham =  set_h_Hamiltonian(model, h = 4)
weight = 5
#rbm.set_evaluate_function(rbm.evaluate_function_dummy(weight))

N = 5000
test = [rbm.expectation_value_batch(Ham, rbm.create_batch(N)) for _ in range(5)]
print(f"The average energy is {np.mean(test)} with standard deviation {np.std(test)}")

batch = rbm.create_batch(N) 
print(f"The spin distributions are {np.average(batch, axis=0)}")

The average energy is (-2.3176+4.157411223423069e-18j) with standard deviation 0.0683838577443536
The spin distributions are [[[0.3604 0.6396]
  [0.8058 0.1942]
  [0.1602 0.8398]]

 [[0.153  0.847 ]
  [0.5682 0.4318]
  [0.353  0.647 ]]]


In [63]:
N = 1000
model = Model(2,3)
rbm = RBM(model)
Ham =  set_h_Hamiltonian(model, h = 4)
#get the current wight and energy
a, b, M = rbm.get_weights()
batch = rbm.create_batch(N)
print(f"the initial energy is {rbm.expectation_value_batch(Ham, batch)}")
print(f"The spin distributions are {np.average(batch, axis=0)}")
delta_a, delta_b, delta_M = rbm.get_deltas(Ham, 1)
gamma = 0.000000001
a -= gamma * delta_a
b -= gamma * delta_b
M -= gamma * delta_M
rbm.set_weights(a, b, M)
batch = rbm.create_batch(N)
print(f"the updated energy is {rbm.expectation_value_batch(Ham, batch)}")
print(f"The spin distributions are {np.average(batch, axis=0)}")

the initial energy is (1.464+1.8055734712594076e-18j)
The spin distributions are [[[0.344 0.656]
  [0.155 0.845]
  [0.757 0.243]]

 [[0.654 0.346]
  [0.726 0.274]
  [0.73  0.27 ]]]
the updated energy is (1.432+3.906405268557802e-18j)
The spin distributions are [[[0.351 0.649]
  [0.15  0.85 ]
  [0.739 0.261]]

 [[0.646 0.354]
  [0.746 0.254]
  [0.726 0.274]]]


In [72]:
N = 1000
model = Model(2,3)
rbm = RBM(model)
Ham =  set_h_Hamiltonian(model, h = 4)
gamma = 10
rbm.train(Ham, gamma, N = 1000, n_iter = 20)

Updating weights at iteration 0
Current energy: (1.528-4.827005079602907e-18j)
Current Sz: 0.382
Updating weights at iteration 1
Current energy: (-3.556+1.887957032453548e-17j)
Current Sz: -0.889
Updating weights at iteration 2
Current energy: (-3.8320000000000003+2.2989299589705544e-19j)
Current Sz: -0.958
Updating weights at iteration 3
Current energy: (-5.032+1.637067109346106e-17j)
Current Sz: -1.258
Updating weights at iteration 4
Current energy: (-6.42+2.966886263448077e-17j)
Current Sz: -1.605
Updating weights at iteration 5
Current energy: (-9.268-8.211223243018583e-19j)
Current Sz: -2.317
Updating weights at iteration 6
Current energy: (-10.9+9.088426713206722e-19j)
Current Sz: -2.725
Updating weights at iteration 7
Current energy: (-11.467999999999998-5.862806675688624e-19j)
Current Sz: -2.867
Updating weights at iteration 8
Current energy: (-11.599999999999998+0j)
Current Sz: -2.9
Updating weights at iteration 9
Current energy: (-11.684000000000003-9.694721481546247e-16j)
Cu

(array([-0.21232647-0.28516707j,  0.0149094 +0.05305621j,
        -0.34748459+0.00143311j, -0.33924761+0.16956976j,
        -0.52620625-0.10107772j, -0.33277799-0.11590467j]),
 array([-0.37662232+0.19905951j, -0.66029705-0.34104079j,
         0.14627137+0.30763765j,  0.10840621+0.32196713j,
         0.45415959+0.18999317j, -0.574677  -0.38886966j,
        -0.10789817+0.28318611j, -0.67960932-0.11285574j,
        -0.29974898+0.51879736j]),
 array([[ 3.07911569e-01-4.89772342e-01j,  3.31794137e-01-1.82413736e-02j,
          1.65736259e-01+1.67792739e-01j,  1.41097830e-01+9.09103918e-02j,
          1.32205434e-01+3.02874981e-01j, -8.32803472e-02-3.16962748e-01j],
        [ 5.73963008e-01+1.44141083e-01j,  2.18187617e-01+1.83149383e-01j,
          8.24606650e-01-2.11291611e-01j,  4.73205928e-01-1.68879482e-01j,
          2.09718852e-01+3.57666818e-01j,  6.03418060e-02+3.70658839e-01j],
        [-3.74543058e-01-4.12346574e-01j, -4.29198412e-01-5.97836289e-01j,
         -3.30871598e-01+2.429

In [73]:
batch = rbm.create_batch(N)
#print(f"the updated energy is {rbm.expectation_value_batch(Ham, batch)}")
print(f"The spin distributions are {np.average(batch, axis=0)}")

The spin distributions are [[[0.    1.   ]
  [0.001 0.999]
  [0.004 0.996]]

 [[0.005 0.995]
  [0.008 0.992]
  [0.    1.   ]]]


### Symmetry of the RBM.

Consider a translation operator working as $\sigma_j(k) = T_k \sigma_j$. An obvious requirement for translation symmetry is that $\Psi_\theta(\sigma) = \Psi_\theta(T_s\sigma)$, and an obvious way to implement this is to just artificially sum over contributions on the output $\sum_i \Psi_\theta(T_{s_i}\sigma)$, but this won't improve the efficiency of the algorithm.

A more tractable wa is to use convolutions. Before we have RBM as 

\begin{align*}
\Psi_M(S;W) &= \sum_{h_i} e^{\sum_j a_j \sigma^z_j + \sum_i b_i h_i + \sum_{ij} W_{ij} h_i \sigma^{z}_j}\\
\end{align*}

Now, we can rewrite it as below, where $f = 1, \alpha_s$ is a number of feature maps, and in particular $W_{j}^{(f)} $ has $\alpha_s \times N$ elements. Note that because the spins are all summed over all translations this is translation invariant.

\begin{equation}
\Psi_{\alpha}(\mathbf{S}; \mathbf{W}) = \sum_{h_{i,s}} \exp \left[ \sum_{f}^{a} \left( a^{(s)} \sum_{s}^{S} \sum_{j}^{N} \tilde{\sigma}_{j}^{z}(s) + b_{f}^{(s)} h_{f,s} + \sum_{s}^{S} \sum_{j}^{N} h_{f,s} W_{j}^{(f)} \tilde{\sigma}_{j}^{z}(s) \right) \right],
\end{equation}
